In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#Question 5
Write code that solves the linear equations required to find vπ(s) and generate the values in the table in Figure 3.2. Note that the policy π picks all valid actions in a state with equal probability. Add comments to your code that explain all your steps.

In [ ]:
action_probability = 1/4
discount = 0.9

initial_grid = np.random.random((5, 5))
#states go from (0,0) to (4,4)

# Special states A and B
A = (0, 1)
A_prime = (4, 1)
B = (0, 3)
B_prime = (2, 3)

actions = ['U', 'D', 'L', 'R']

grid_size = 5

def next_state_and_reward(state, action):
    row, col = state

    if state == A:
        return (A_prime, 10)  # Move to A_prime with a reward of +10
    if state == B:
        return (B_prime, 5)   # Move to B_prime with a reward of +5

    elif action == 'U':
        return (max(row - 1, 0), col), -(row == 0)
    elif action == 'D':
        return (min(row + 1, 4), col), -(row == 4)
    elif action == 'L':
        return (row, max(col - 1, 0)), -(col == 0)
    elif action == 'R':
        return (row, min(col + 1, 4)), -(col == 4)

In [ ]:

# Policy Evaluation
def policy_evaluation(value_function, discount, action_probability):
    theta=1e-4
    for i in range(1000):
        delta = 0

        for i in range(grid_size):
            for j in range(grid_size):
                state = (i, j)
                v = value_function[state]

                new_value = 0
                for action in actions:
                    (next_state, reward) = next_state_and_reward(state, action)
                    new_value += action_probability * (reward + discount * value_function[next_state])

                value_function[state] = new_value

                delta = max(delta, abs(v - new_value))

        if delta < theta:
            break

    return value_function

# Run policy evaluation
value_function = policy_evaluation(value_function, discount, action_probability)

# Display the value function
print("State-Value Function:")
print(np.round(value_function, 1))


State-Value Function:
[[ 3.3  8.8  4.4  5.3  1.5]
 [ 1.5  3.   2.3  1.9  0.5]
 [ 0.1  0.7  0.7  0.4 -0.4]
 [-1.  -0.4 -0.4 -0.6 -1.2]
 [-1.9 -1.3 -1.2 -1.4 -2. ]]


#Question 7
Write code that generates the optimal state-value function and the optimal policy for the
Gridworld in Figure 3.5. You want to solve the corresponding system of non-linear equations. Explain all your
steps.

In [ ]:
action_probability = 1/4
discount = 0.9

initial_grid = np.random.random((5, 5))
#states go from (0,0) to (4,4)

# Special states A and B
A = (0, 1)
A_prime = (4, 1)
B = (0, 3)
B_prime = (2, 3)

actions = ['U', 'D', 'L', 'R']

grid_size = 5

def next_state_and_reward(state, action):
    row, col = state

    if state == A:
        return (A_prime, 10)  # Move to A_prime with a reward of +10
    if state == B:
        return (B_prime, 5)   # Move to B_prime with a reward of +5

    elif action == 'U':
        return (max(row - 1, 0), col), -(row == 0)
    elif action == 'D':
        return (min(row + 1, 4), col), -(row == 4)
    elif action == 'L':
        return (row, max(col - 1, 0)), -(col == 0)
    elif action == 'R':
        return (row, min(col + 1, 4)), -(col == 4)


In [ ]:

# Value Iteration algorithm to find optimal state-value function and policy
def value_iteration(theta=1e-4, discount=0.9):
    value_function = np.zeros((grid_size, grid_size))  # Initialize values to 0
    policy = np.full((grid_size, grid_size), '', dtype=object)  # Initialize an empty policy

    iteration = 0
    while True:
        delta = 0
        # Iterate over all states in the grid
        for i in range(grid_size):
            for j in range(grid_size):
                state = (i, j)
                v = value_function[state]

                max_value = float('-inf')
                best_actions = []
                for action in actions:
                    (next_state, reward) = next_state_and_reward(state, action)
                    value = reward + discount * value_function[next_state]


                    if value > max_value:
                        max_value = value
                        best_actions = [action]
                    elif value == max_value:
                        best_actions.append(action)

                # Updating the state-value function and policy
                value_function[state] = max_value
                policy[state] = ''.join(best_actions)


                delta = max(delta, abs(v - value_function[state]))

        iteration += 1
        if delta < theta:
            break

    return value_function, policy


optimal_value_function, optimal_policy = value_iteration()


print("Optimal Value Function:")
print(np.round(optimal_value_function, 1))

print("\nOptimal Policy:")
for row in optimal_policy:
    print(' '.join(row))

Optimal Value Function:
[[22.  24.4 22.  19.4 17.5]
 [19.8 22.  19.8 17.8 16. ]
 [17.8 19.8 17.8 16.  14.4]
 [16.  17.8 16.  14.4 13. ]
 [14.4 16.  14.4 13.  11.7]]

Optimal Policy:
R UDLR L UDLR L
UR U UL L L
UR U UL UL UL
UR U UL UL UL
UR U UL UL UL


#Question 9.
Code policy iteration and value iteration (VI) to solve the Gridworld in Example 4.1. Your code
must log output of each iteration. Pick up a few sample iterations to show policy evaluation and improvement at
work. Similarly, show using a few obtained iterations that every iteration of VI improves the value function. Your
code must include the fix to the bug mentioned in Exercise 4.4.

In [ ]:
discount = 1.0

initial_grid = np.random.random((4, 4))

terminal_states = [(0, 0), (3, 3)]
actions = ['U', 'D', 'L', 'R']
reward = -1
grid_size = 4
theta = 1e-4
# equiprobable random policy
action_probability = 1/4

# policy and value function
policy = np.random.choice(actions, size=(grid_size, grid_size))
value_function = np.zeros((grid_size, grid_size))

def next_state_and_reward(state, action):
    row, col = state


    if action == 'U':
        return (max(row - 1, 0), col), -1
    elif action == 'D':
        return (min(row + 1, 3), col), -1
    elif action == 'L':
        return (row, max(col - 1, 0)), -1
    elif action == 'R':
        return (row, min(col + 1, 3)), -1


In [ ]:
# Policy Evaluation
def policy_evaluation(policy, value_function):
    for iter in range(100):
        for i in range(grid_size):
            for j in range(grid_size):
                state = (i, j)
                if state in terminal_states:
                    continue  # Skip terminal states

                v = value_function[state]

                next_state, _ = next_state_and_reward(state, policy[state])
                value_function[state] = reward + discount * value_function[next_state]

    return value_function

# Policy Improvement
def policy_improvement(policy, value_function):
    policy_stable = True
    for i in range(grid_size):
        for j in range(grid_size):
            state = (i, j)
            if state in terminal_states:
                continue

            old_action = policy[state]

            action_values = []
            for action in actions:
                next_state, _ = next_state_and_reward(state, action)
                action_values.append(reward + discount * value_function[next_state])


            best_action = actions[np.argmax(action_values)]
            policy[state] = best_action


            if best_action != old_action:
                policy_stable = False
    return policy, policy_stable

# Policy Iteration
def policy_iteration():
    # Initialize value function and random policy
    value_function = np.zeros((grid_size, grid_size))
    policy = np.random.choice(actions, size=(grid_size, grid_size))

    for iteration in range(100):
        print(f"Policy Iteration {iteration}:")
        print("Policy:")
        print(policy)
        print("Value Function:")
        print(value_function)

        value_function = policy_evaluation(policy, value_function)

        policy, policy_stable = policy_improvement(policy, value_function)

        print(policy, value_function)

        if policy_stable:
            break

    return policy, value_function

final_policy, final_value_function = policy_iteration()

print("\nFinal Policy:")
print(final_policy)
print("\nFinal Value Function:")
print(final_value_function)


Policy Iteration 0:
Policy:
[['D' 'R' 'L' 'R']
 ['D' 'D' 'D' 'D']
 ['D' 'D' 'D' 'D']
 ['L' 'L' 'R' 'L']]
Value Function:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[['D' 'L' 'D' 'D']
 ['U' 'R' 'D' 'D']
 ['U' 'R' 'D' 'D']
 ['U' 'R' 'R' 'L']] [[   0. -199. -200. -100.]
 [-100. -101.   -3.   -2.]
 [-100. -101.   -2.   -1.]
 [-100. -101.   -1.    0.]]
Policy Iteration 1:
Policy:
[['D' 'L' 'D' 'D']
 ['U' 'R' 'D' 'D']
 ['U' 'R' 'D' 'D']
 ['U' 'R' 'R' 'L']]
Value Function:
[[   0. -199. -200. -100.]
 [-100. -101.   -3.   -2.]
 [-100. -101.   -2.   -1.]
 [-100. -101.   -1.    0.]]
[['D' 'L' 'L' 'D']
 ['U' 'U' 'D' 'D']
 ['U' 'D' 'D' 'D']
 ['U' 'R' 'R' 'L']] [[ 0. -1. -4. -3.]
 [-1. -4. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Policy Iteration 2:
Policy:
[['D' 'L' 'L' 'D']
 ['U' 'U' 'D' 'D']
 ['U' 'D' 'D' 'D']
 ['U' 'R' 'R' 'L']]
Value Function:
[[ 0. -1. -4. -3.]
 [-1. -4. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
[['D' 'L' 'L' 'D']
 ['U' 'U' 'U' 'D']
 ['U' 'U' 'D

In [ ]:
value_function = np.zeros((grid_size, grid_size))
policy = np.random.choice(actions, size=(grid_size, grid_size))

# Value Iteration
def value_iteration(value_function):
    while True:
        delta = 0
        for i in range(grid_size):
            for j in range(grid_size):
                state = (i, j)
                if state in terminal_states:
                    continue

                v = value_function[state]

                action_values = []
                for action in actions:
                    next_state, _ = next_state_and_reward(state, action)
                    action_values.append(reward + discount * value_function[next_state])

                value_function[state] = max(action_values)

                delta = max(delta, abs(v - value_function[state]))

        print(f"Value function: {value_function}")
        if delta < theta:
            break

    return value_function

# Derive policy
def derive_policy(value_function):
    policy = np.zeros((grid_size, grid_size), dtype=str)
    for i in range(grid_size):
        for j in range(grid_size):
            state = (i, j)
            if state in terminal_states:
                policy[state] = 'T'
                continue

            action_values = []
            for action in actions:
                next_state, _ = next_state_and_reward(state, action)
                action_values.append(reward + discount * value_function[next_state])

            best_action = actions[np.argmax(action_values)]
            policy[state] = best_action

    return policy

final_value_function = value_iteration(value_function)

optimal_policy = derive_policy(final_value_function)


print("\nFinal Value Function:")
print(final_value_function)
print("\nOptimal Policy:")
print(optimal_policy)

Va;ue function: [[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]
Va;ue function: [[ 0. -1. -2. -2.]
 [-1. -2. -2. -2.]
 [-2. -2. -2. -1.]
 [-2. -2. -1.  0.]]
Va;ue function: [[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Va;ue function: [[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Final Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Optimal Policy:
[['T' 'L' 'L' 'D']
 ['U' 'U' 'U' 'D']
 ['U' 'U' 'D' 'D']
 ['U' 'R' 'R' 'T']]
